# Challenge 3: Standard Enhancement Multi-Agent System

## Overview
This notebook implements the multi-agent AI system for reviewing, suggesting, and validating updates to AAOIFI standards as specified in Challenge 3 of the hackathon.

We'll focus on developing three specialized agents:
1. **Review & Extraction Agent**: Processes the selected standard to extract key elements, definitions, principles, scope, recognition criteria, measurement rules, and disclosure requirements.
2. **Enhancement Agent**: Suggests modifications/enhancements by comparing with other standards, analyzing contemporary financial instruments, and identifying improvement areas.
3. **Validation Agent**: Validates proposed changes for compliance with AAOIFI principles and Shariah requirements.

For this implementation, we'll focus on **FAS 10** (Istisna'a and Parallel Istisna'a).

## 1. Environment Setup

First, let's import the necessary libraries and initialize our environment.

In [ ]:
# Standard imports
import os
import sys
import json
from pathlib import Path
from typing import List, Dict, Any, Optional

# LangChain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools import BaseTool
from langchain.schema import BaseMessage, AIMessage, HumanMessage

# Configure paths
project_root = Path(r"c:\Users\Tayeb Kahia\Desktop\ISDB")
data_dir = project_root / "data"
vector_db_dir = project_root / "vector_db" / "standards_enhancement"

print("Setup complete!")

## 2. PDF Processing and Vector DB Setup

We'll process the PDF files for the standards and create a vector database for efficient retrieval.

In [ ]:
def process_pdf(pdf_path):
    """Load and split a PDF into chunks."""
    # Load PDF
    print(f"Loading {pdf_path}...")
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    
    # Extract metadata from filename
    filename = os.path.basename(pdf_path)
    standard_type = "FAS" if "FAS" in filename else "SS"
    standard_num = "".join(filter(str.isdigit, filename))
    
    # Add metadata to pages
    for page in pages:
        page.metadata["standard_type"] = standard_type
        page.metadata["standard_num"] = standard_num
        page.metadata["source"] = filename
        # Keep existing page number metadata
    
    # Split into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", ".", "!", "?", ";", ",", " ", ""]
    )
    chunks = text_splitter.split_documents(pages)
    print(f"Split {len(pages)} pages into {len(chunks)} chunks")
    
    return chunks

def create_vector_db():
    """Process PDF files and create a vector database."""
    # Create vector DB directory if it doesn't exist
    vector_db_dir.mkdir(parents=True, exist_ok=True)
    
    # Check if vector DB already exists
    if os.path.exists(vector_db_dir / "chroma.sqlite3"):
        print("Vector DB already exists. Loading existing DB...")
        # Setup embeddings
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        # Load existing DB
        vector_db = Chroma(embedding_function=embeddings, persist_directory=str(vector_db_dir))
        return vector_db
    
    # Process standard files (focusing on FAS 10 and related SS)
    pdf_files = [
        data_dir / "FAS10.PDF",
        data_dir / "SS11.pdf",  # Related Shariah Standard
        data_dir / "FAS32.pdf",  # For comparison
        data_dir / "FAS4.PDF"    # For comparison
    ]
    
    # Process each PDF and collect all chunks
    all_chunks = []
    for pdf_file in pdf_files:
        if pdf_file.exists():
            chunks = process_pdf(str(pdf_file))
            all_chunks.extend(chunks)
        else:
            print(f"Warning: {pdf_file} not found")
    
    if not all_chunks:
        raise ValueError("No documents were processed. Check file paths.")
    
    # Setup embeddings and create vector DB
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma.from_documents(
        documents=all_chunks,
        embedding=embeddings,
        persist_directory=str(vector_db_dir)
    )
    vector_db.persist()
    print(f"Created vector DB with {len(all_chunks)} chunks")
    
    return vector_db

# Create or load the vector database
vector_db = create_vector_db()

## 3. Define the Multi-Agent System

Let's implement the three specialized agents for our system.

In [ ]:
# Initialize LLM
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0)

# Create a retriever from the vector DB
retriever = vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}  # Retrieve more documents for comprehensive analysis
)

### 3.1 Agent 1: Review & Extraction Agent

In [ ]:
# Define the Review & Extraction Agent

review_template = """
You are a specialized AI agent for reviewing and extracting key information from AAOIFI Financial Accounting Standards (FAS).
Your task is to comprehensively analyze FAS 10 (Istisna'a and Parallel Istisna'a) and extract its key elements.

Based on the following context information about the standard, please extract and organize the following elements:

Context information:
{context}

Please extract and organize the following:

1. Definitions of key terms in FAS 10
2. Scope and applicability of the standard
3. Core principles and concepts
4. Recognition criteria for Istisna'a contracts
5. Measurement rules and methodologies
6. Disclosure requirements
7. Areas that are potentially unclear or ambiguous
8. Special cases or exceptions mentioned

Format your response as a detailed structured analysis that another AI agent could use to propose improvements.
"""

# Create the RAG pipeline for the Review Agent
review_prompt = ChatPromptTemplate.from_template(review_template)
review_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | review_prompt
    | llm
    | StrOutputParser()
)

# Function to run the review agent
def run_review_agent(standard="FAS 10 Istisna'a and Parallel Istisna'a"):
    """Run the review agent to extract information from the standard."""
    query = f"Analyze and extract key elements from {standard}"
    return review_chain.invoke(query)

### 3.2 Agent 2: Enhancement Agent

In [ ]:
# Define the Enhancement Agent

enhancement_template = """
You are a specialized AI agent for suggesting improvements to AAOIFI Financial Accounting Standards (FAS).
Your task is to analyze the extracted information from FAS 10 (Istisna'a and Parallel Istisna'a) and suggest meaningful enhancements.

Standard information extracted by the Review Agent:
{standard_info}

Additional context from related standards:
{context}

Please propose enhancements in the following areas:

1. Clarifications for ambiguous or unclear sections
2. Additional examples or illustrations for complex concepts
3. Updates to align with contemporary financial instruments or practices
4. Potential harmonization with global standards (like IFRS) while maintaining Shariah compliance
5. Additional guidance for practical implementation
6. Improvements to disclosure requirements
7. Addressing gaps in coverage of specific transaction types
8. Enhanced structure or organization for better usability

For each enhancement suggestion:
- Clearly identify the specific section/aspect of FAS 10 being addressed
- Explain the current limitation or issue
- Provide the proposed enhancement with specific wording where appropriate
- Justify why this enhancement would improve the standard's usability or effectiveness

IMPORTANT: Ensure all suggestions maintain Shariah compliance and align with Islamic financial principles.
"""

# Create the RAG pipeline for the Enhancement Agent
enhancement_prompt = ChatPromptTemplate.from_template(enhancement_template)
enhancement_chain = (
    {"context": retriever, "standard_info": RunnablePassthrough(), "question": lambda x: "Comparative analysis of FAS 10 with other standards"}
    | enhancement_prompt
    | llm
    | StrOutputParser()
)

# Function to run the enhancement agent
def run_enhancement_agent(standard_info):
    """Run the enhancement agent to suggest improvements to the standard."""
    return enhancement_chain.invoke(standard_info)

### 3.3 Agent 3: Validation Agent

In [ ]:
# Define the Validation Agent

validation_template = """
You are a specialized AI agent for validating proposed enhancements to AAOIFI Financial Accounting Standards (FAS).
Your task is to evaluate the suggested improvements to FAS 10 (Istisna'a and Parallel Istisna'a) for compliance with Shariah principles and AAOIFI's overall framework.

Standard information extracted by the Review Agent:
{standard_info}

Proposed enhancements from the Enhancement Agent:
{enhancements}

Additional context from Shariah standards:
{context}

Please validate each proposed enhancement by assessing:

1. Shariah Compliance: Does the enhancement maintain or improve compliance with Islamic financial principles?
2. Consistency: Is it consistent with AAOIFI's overall framework and other standards?
3. Practicality: Would it be practical to implement in real-world Islamic financial institutions?
4. Clarity: Does it enhance or maintain the clarity of the standard?
5. Relevance: Is it relevant to contemporary Islamic finance practices?
6. Thoroughness: Does it address the identified issue comprehensively?

For each enhancement:
- Provide a rating (Approved, Approved with Modifications, or Rejected)
- Justify your decision with specific references to Shariah principles or AAOIFI requirements where applicable
- For 'Approved with Modifications', suggest the specific modifications needed
- For 'Rejected', explain the specific issues that make the enhancement inappropriate

Conclude with a summary of which enhancements should be adopted, modified, or rejected.
"""

# Create the RAG pipeline for the Validation Agent
validation_prompt = ChatPromptTemplate.from_template(validation_template)
validation_chain = (
    {"context": retriever, "standard_info": RunnablePassthrough(), "enhancements": RunnablePassthrough(), "question": lambda x: "Shariah compliance of FAS 10 proposed enhancements"}
    | validation_prompt
    | llm
    | StrOutputParser()
)

# Function to run the validation agent
def run_validation_agent(standard_info, enhancements):
    """Run the validation agent to evaluate proposed enhancements."""
    return validation_chain.invoke({"standard_info": standard_info, "enhancements": enhancements})

## 4. Multi-Agent Orchestration

Now we'll implement the orchestration logic to coordinate our agents.

In [ ]:
def run_multi_agent_system():
    """Run the complete multi-agent system for standard enhancement."""
    print("Starting Multi-Agent System for FAS 10 Standard Enhancement...\n")
    
    # Step 1: Run the Review & Extraction Agent
    print("AGENT 1 (Review & Extraction): Analyzing FAS 10...")
    standard_info = run_review_agent()
    print("\nReview & Extraction Complete!\n")
    
    # Step 2: Run the Enhancement Agent
    print("AGENT 2 (Enhancement): Suggesting improvements to FAS 10...")
    enhancements = run_enhancement_agent(standard_info)
    print("\nEnhancement Suggestions Complete!\n")
    
    # Step 3: Run the Validation Agent
    print("AGENT 3 (Validation): Evaluating proposed enhancements for Shariah compliance...")
    validation_results = run_validation_agent(standard_info, enhancements)
    print("\nValidation Complete!\n")
    
    # Return all results
    results = {
        "standard_info": standard_info,
        "enhancements": enhancements,
        "validation_results": validation_results
    }
    
    # Save results to file
    with open(project_root / "notebooks" / "fas10_enhancement_results.json", "w") as f:
        json.dump(results, f, indent=2)
    
    return results

## 5. Run the Multi-Agent System

Let's execute our multi-agent system for FAS 10 enhancement.

In [ ]:
# Run the full multi-agent system
results = run_multi_agent_system()

# Display summary of results
print("\n==== MULTI-AGENT SYSTEM RESULTS ====\n")
print("Standard Information Extracted by Agent 1:\n")
print(results["standard_info"][:500] + "...\n\n")

print("Enhancements Proposed by Agent 2:\n")
print(results["enhancements"][:500] + "...\n\n")

print("Validation Results from Agent 3:\n")
print(results["validation_results"][:500] + "...\n")

print("\nFull results saved to 'fas10_enhancement_results.json'")

## 6. Interactive Query System

Now let's create a simple interactive system to query our enhanced standard.

In [ ]:
query_template = """
You are an AI assistant specialized in AAOIFI standards, particularly FAS 10 (Istisna'a and Parallel Istisna'a).
Use the following information to answer the user's question:

Standard Information:
{standard_info}

Proposed Enhancements:
{enhancements}

Validation Results:
{validation_results}

Additional Context from Standards:
{context}

User Question: {question}

Provide a detailed, accurate response. If the user's question relates to a proposed enhancement,
make sure to indicate whether that enhancement was approved or not during validation.
"""

# Create a query chain
query_prompt = ChatPromptTemplate.from_template(query_template)
query_chain = (
    {"context": retriever, "standard_info": lambda x: results["standard_info"], 
     "enhancements": lambda x: results["enhancements"], 
     "validation_results": lambda x: results["validation_results"],
     "question": RunnablePassthrough()}
    | query_prompt
    | llm
    | StrOutputParser()
)

def query_enhanced_standard(question):
    """Query the enhanced standard with a specific question."""
    return query_chain.invoke(question)

In [ ]:
# Example queries
questions = [
    "What are the main improvements suggested for FAS 10?",
    "How does FAS 10 handle revenue recognition in Istisna'a contracts?",
    "What challenges exist in the current standard regarding parallel Istisna'a?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    answer = query_enhanced_standard(question)
    print(f"\nAnswer: {answer}\n")
    print("-" * 80)

## 7. Conclusion and Next Steps

Our multi-agent system successfully analyzes FAS 10, proposes enhancements, and validates them against Shariah principles. This approach demonstrates how AI can assist in improving Islamic finance standards while ensuring compliance with fundamental principles.

Potential next steps include:
1. Extending the system to handle other AAOIFI standards (FAS 4, FAS 32, etc.)
2. Improving the retrieval system with more sophisticated chunking and embedding approaches
3. Adding more specialized agents (e.g., for specific aspects like translation, examples generation)
4. Creating a visual interface for interacting with the system
5. Implementing a continuous improvement mechanism that learns from user feedback